In [1]:
# from PDFDataExtractor.pdfdataextractor import Reader
import pdfplumber 
import fitz

import re
pdf_dir = r'C:\Users\calvin.howard\OneDrive\OneDrive_Documents\Research\_memory_and_ad_dbs\Literature\distinct_ad_atrophy_resultds_in_distinct_impairment.pdf'

In [32]:
#Acceptable Functions
def extract_without_page_headers_and_footers(pdf_file, margin_top=100, margin_bottom=100):
    text = ""
    with pdfplumber.open(pdf_file) as pdf:
        for page in pdf.pages:
            words = page.extract_words()
            for word in words:
                if margin_top < word['top'] < (page.height - margin_bottom):
                    text += word['text'] + ' '
            text += "\n"
    return text

def extract_without_page_margins(pdf_file, margin_left=50, margin_top=50, margin_right=550, margin_bottom=750):
    text = ""
    with pdfplumber.open(pdf_file) as pdf:
        for page in pdf.pages:
            page_width = page.width
            page_height = page.height
            if margin_right is None:
                margin_right = page_width
            if margin_bottom is None:
                margin_bottom = page_height
            words = page.crop((margin_left, margin_top, margin_right, margin_bottom)).extract_words()
            for word in words:
                text += word['text'] + ' '
            text += "\n"
    return text


def remove_reference_sections(text):
    # Regular expression pattern to match common reference section headers
    pattern = r'\b(?:references|citations|bibliography)\b'
    
    # Find the reference section
    reference_section_start = re.search(pattern, text)
    
    # If a reference section is found, remove it and the text after it
    if reference_section_start:
        print('found it')
        text = text[:reference_section_start.start()]
    return text

def add_newlines_around_all_capitalized_words(text):
    pattern = r'\b([A-Z]{5,}(?:\s+[A-Z]{5,})*)\b'
    replacement = r'\n\1\n'
    formatted_text = re.sub(pattern, replacement, text)
    return formatted_text

def detect_columns(page):
    # Extract words from the page
    words = page.extract_words()

    # Group words by their X coordinate
    word_groups = {}
    for word in words:
        x = round(word["x0"])
        if x not in word_groups:
            word_groups[x] = []
        word_groups[x].append(word)

    # Sort the groups by X coordinate
    sorted_word_groups = sorted(word_groups.items(), key=lambda x: x[0])

    # Compute the spacing between groups
    spacings = []
    for i in range(len(sorted_word_groups) - 1):
        spacing = sorted_word_groups[i + 1][0] - sorted_word_groups[i][0]
        spacings.append(spacing)

    # Compute the median spacing
    median_spacing = sorted(spacings)[len(spacings) // 2]

    # Group columns based on the spacing
    columns = []
    current_column = []
    for i in range(len(sorted_word_groups)):
        current_column.append(sorted_word_groups[i][1])
        if i == len(sorted_word_groups) - 1 or sorted_word_groups[i + 1][0] - sorted_word_groups[i][0] > median_spacing:
            columns.append(current_column)
            current_column = []

    return columns


In [33]:

import pdfplumber
from collections import defaultdict
import re

def preprocess_text(text):
    # Add space after punctuation
    text = re.sub(r'([,.:;])', r'\1 ', text)

    # Add space before and after parentheses
    text = re.sub(r'([(])', r' \1', text)
    text = re.sub(r'([)])', r'\1 ', text)

    # Add space before and after hyphens, but not within words containing hyphens
    text = re.sub(r'(?<![a-zA-Z0-9-])-(?![a-zA-Z0-9-])', r' - ', text)

    # Remove unnecessary hyphenation at line breaks
    text = re.sub(r'-\s+', '', text)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

def extract_without_page_margins(pdf_file, n_columns, margin_left=50, margin_top=100, margin_right=550, margin_bottom=725, space_threshold=0.00000001):
    text = ""
    with pdfplumber.open(pdf_file) as pdf:
        for page in pdf.pages:
            page_width = page.width
            page_height = page.height
            if margin_right is None:
                margin_right = page_width
            if margin_bottom is None:
                margin_bottom = page_height
            cropped_page = page.crop((margin_left, margin_top, margin_right, margin_bottom))

            column_width = (margin_right - margin_left) / n_columns
            for idx in range(n_columns):
                col_left = margin_left + idx * column_width
                col_right = col_left + column_width
                cropped_col = cropped_page.crop((col_left, margin_top, col_right, margin_bottom))
                col_words = cropped_col.extract_words()
                lines = defaultdict(list)
                for word in col_words:
                    line_key = round(word['top'], 2)
                    lines[line_key].append(word)

                sorted_lines = sorted(lines.items(), key=lambda x: x[0], reverse=True)
                text_lines = []
                for line in sorted_lines:
                    sorted_words = sorted(line[1], key=lambda x: x['x0'])
                    line_text = sorted_words[0]['text']
                    for i in range(1, len(sorted_words)):
                        prev_word = sorted_words[i - 1]
                        curr_word = sorted_words[i]
                        if curr_word['x0'] - prev_word['x1'] > space_threshold:
                            line_text += ' '
                        line_text += curr_word['text']
                    text_lines.append(line_text)

                text += ' '.join(text_lines) + '\n'
    return text




pdf_file = pdf_dir
extracted_text = extract_without_page_margins(pdf_file, n_columns=2)
formatted_text = add_newlines_around_all_capitalized_words(extracted_text)
formatted_text = formatted_text.lower()
formatted_text = remove_reference_sections(formatted_text)
formatted_text = preprocess_text(formatted_text)

with open("output.txt", "w", encoding="utf-8") as output_file:
    output_file.write(formatted_text)

In [95]:
#Works nicely, fails to detect columns
pdf_file = pdf_dir
extracted_text = extract_without_page_margins(pdf_file)
formatted_text = add_newlines_around_all_capitalized_words(extracted_text)
formatted_text = formatted_text.lower()
formatted_text = remove_reference_sections(formatted_text)

with open("output.txt", "w", encoding="utf-8") as output_file:
    output_file.write(formatted_text)

TypeError: extract_without_page_margins() missing 1 required positional argument: 'n_columns'

In [30]:
# Testing updates
#----------------------------------------------------------------PREPROCESSING 
def preprocess_text(text):
    # Add space after punctuation
    text = re.sub(r'([,.:;])', r'\1 ', text)

    # Add space before and after parentheses
    text = re.sub(r'([(])', r' \1', text)
    text = re.sub(r'([)])', r'\1 ', text)

    # Add space before and after hyphens, but not within words containing hyphens
    text = re.sub(r'(?<![a-zA-Z0-9-])-(?![a-zA-Z0-9-])', r' - ', text)

    # Remove unnecessary hyphenation at line breaks
    text = re.sub(r'-\s+', '', text)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

def remove_urls(text):
    url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    return url_pattern.sub('', text)

def remove_email_addresses(text):
    email_pattern = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b')
    return email_pattern.sub('', text)

def full_preprocess(text):
    text = add_newlines_around_all_capitalized_words(text)
    text = text.lower()
    # text = remove_reference_sections(text)
    text = preprocess_text(text)
    # text = remove_urls(text)
    # text = remove_email_addresses(text)
    return text

##---------------------------------------------------------------- EXTRACTION
def extract_without_page_margins_v2(pdf_file, margin_left=0, margin_top=0, margin_right=None, margin_bottom=None, space_threshold=0.00000001):
    text = ""
    with pdfplumber.open(pdf_file) as pdf:
        for page in pdf.pages:
            page_width = page.width
            page_height = page.height
            if margin_right is None:
                margin_right = page_width
            if margin_bottom is None:
                margin_bottom = page_height
            cropped_page = page.crop((margin_left, margin_top, margin_right, margin_bottom))

            columns = detect_columns(cropped_page)
            for column_words in columns:
                col_words = [word for group in column_words for word in group]
                lines = {}
                for word in col_words:
                    line_key = round(word['top'], 2)
                    if line_key not in lines:
                        lines[line_key] = []
                    lines[line_key].append(word)

                sorted_lines = sorted(lines.items(), key=lambda x: x[0], reverse=True)
                text_lines = []
                for line in sorted_lines:
                    sorted_words = sorted(line[1], key=lambda x: x['x0'])
                    line_text = sorted_words[0]['text']
                    for i in range(1, len(sorted_words)):
                        prev_word = sorted_words[i - 1]
                        curr_word = sorted_words[i]
                        if curr_word['x0'] - prev_word['x1'] > space_threshold:
                            line_text += ' '
                        line_text += curr_word['text']
                    text_lines.append(line_text)

                text += '\n'.join(text_lines) + '\n'
    return text

# The rest of your code remains the same





def detect_columns(page):
    # Extract words from the page
    words = page.extract_words()

    # Group words by their X coordinate
    word_groups = {}
    for word in words:
        x = round(word["x0"])
        if x not in word_groups:
            word_groups[x] = []
        word_groups[x].append(word)

    # Sort the groups by X coordinate
    sorted_word_groups = sorted(word_groups.items(), key=lambda x: x[0])

    # Compute the spacing between groups
    spacings = []
    for i in range(len(sorted_word_groups) - 1):
        spacing = sorted_word_groups[i + 1][0] - sorted_word_groups[i][0]
        spacings.append(spacing)

    # Compute the median spacing
    median_spacing = sorted(spacings)[len(spacings) // 2]

    # Group columns based on the spacing
    columns = []
    current_column = []
    for i in range(len(sorted_word_groups)):
        current_column.append(sorted_word_groups[i][1])
        if i == len(sorted_word_groups) - 1 or sorted_word_groups[i + 1][0] - sorted_word_groups[i][0] > median_spacing:
            columns.append(current_column)
            current_column = []

    return columns



pdf_file = pdf_dir
extracted_text = extract_without_page_margins(pdf_file, n_columns=2)
formatted_text = full_preprocess(extracted_text)

with open("output.txt", "w", encoding="utf-8") as output_file:
    output_file.write(formatted_text)

## Attempt with textract

In [11]:
import os
import textract

def extract_text_from_pdf(file_path):
    text = textract.process(file_path)
    return text.decode("utf-8")

def save_text_to_file(text, output_file_path):
    with open(output_file_path, "w", encoding="utf-8") as output_file:
        output_file.write(text)

def extract_text_from_pdf_dir(pdf_dir, output_dir):
    for file_name in os.listdir(pdf_dir):
        if file_name.endswith(".pdf"):
            input_file_path = os.path.join(pdf_dir, file_name)
            output_file_path = os.path.join(output_dir, f"{os.path.splitext(file_name)[0]}.txt")
            text = extract_text_from_pdf(input_file_path)
            save_text_to_file(text, output_file_path)

pdf_dir = r'C:\Users\calvin.howard\OneDrive\OneDrive_Documents\Research\_memory_and_ad_dbs\Literature'
output_dir = r"C:\Users\calvin.howard\OneDrive\OneDrive_Documents\Work\PostDoc\Nimlab\software_env\python_modules\nimlab\gpt_document_reader\writes"
extract_text_from_pdf_dir(pdf_dir, output_dir)



## Check out tesseract. Also seems extremely powerful. 